# Augenblick and Rabin, 2019, "An Experiment on Time Preference and Misprediction in Unpleasant Tasks", Table 1

#### Authors:  

- Massimiliano Pozzi (Bocconi University, pozzi.massimiliano@studbocconi.it)
- Salvatore Nunnari (Bocconi University, salvatore.nunnari@unibocconi.it)
- Adaptations by Richard Foltyn (NHH, richard.foltyn@nhh.no)

#### Description:

The code in this Jupyter notebook performs the aggregate estimates to replicate column 1 of Table 1

This notebook was tested with the following packages versions (also see the `environment.yml` file):
- Python 3.11, numpy 1.26, scipy 1.11, pandas 2.0, autograd 1.6

In [1]:
# Import the necessary libraries

from autograd.scipy.stats import norm    
import autograd.numpy as anp
import pandas as pd
import scipy.optimize as opt
import autograd
import numpy as np

## 1. Data Cleaning and Data Preparation

We import the dataset containing the choices of all 100 individuals who participated to the experiment. To guarantee consistency with the authors' results, we then construct the primary sample used for the aggregate estimates. This sample consists of 72 individuals whose individual parameter estimates converged in less than 200 iterations when using the authors' Stata algorithm. In particular, we run the `03MergeIndMLEAndConstructMainSample.do` file provided by the authors. This script creates a file named `ind_to_keep.csv` which contains the identifiers of the individuals to keep.

In [2]:
# Import the two datasets and drop subjects whose individual estimates do not converge

dt = pd.read_stata('../input/decisions_data.dta')    # full sample
ind_keep = pd.read_csv('../input/ind_to_keep.csv')   # import csv with ID of subjects to keep 

# drop subjects whose IDs are not listed in the ind_keep dataframe (28 individuals)

dt = dt[dt.wid.isin(ind_keep.wid_col1)] # this is the primary sample for the aggregate estimates (72 individuals)

We remove observations when a bonus was offered and create the following dummy variables that will be useful for estimation: `pb` is equal to one if the subject completed 10 mandatory tasks on subject-day (this is used to estimate the projection bias parameter $\alpha$); `ind_effort10` and `ind_effort110` are equal to one if, respectively, the subject completed 10 or 110 tasks (and they are used for the Tobit correction when computing the likelihood).

In [3]:
# Remove observations when a bonus was offered and create dummy variables. 

dt = dt[dt.bonusoffered !=1].copy()   # remove observations when a bonus was offered
dt['pb']= dt['workdone1']/10   # pb dummy variable. workdone1 can either be 10 or 0, so dividing the variable by 10 creates our dummy
dt['ind_effort10']  = (dt['effort']==10).astype(int)   # ind_effort10 dummy
dt['ind_effort110'] = (dt['effort']==110).astype(int)  # ind_effort110 dummy
dt = dt.reset_index(drop=True) # correct the index. The index should go from 0 to 8048

In [4]:
# Convert individual ID stored in wid to integer
dt["wid"] = dt["wid"].astype(int)

In [5]:
# Keep only variables relevant for estimation
keep = ['wid', 'netdistance', 'wage', 'today', 'prediction', 'pb', 'effort', 'ind_effort10', 'ind_effort110']
dt = dt[keep].copy()
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8049 entries, 0 to 8048
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   wid            8049 non-null   int64  
 1   netdistance    8049 non-null   float32
 2   wage           8049 non-null   float32
 3   today          8049 non-null   float32
 4   prediction     8049 non-null   float32
 5   pb             8049 non-null   float64
 6   effort         8049 non-null   float32
 7   ind_effort10   8049 non-null   int64  
 8   ind_effort110  8049 non-null   int64  
dtypes: float32(5), float64(1), int64(3)
memory usage: 408.9 KB


## 2. Define the Model and the Likelihood (Section 3 in Paper)

The agent needs to choose the optimal effort $e$ to solve a simple tradeoff problem between disutility of effort and consumption utility derived from the consequent payment. More specifically, the agent takes a decision at a time $k$ to complete a certain number of tasks at time $t\geq k$ and to get paid a wage $w$ per task at time $T > k$. 
Assuming the agent discounts utility using quasi-hyperbolic discounting and has a convex cost function $C(e)$, the problem can be conveniently written as:

$$ \max_{{e}} \; \delta^{T-k}⋅(e⋅w)- \frac{1}{\beta^{I(k=t)}}⋅\frac{1}{\beta_h^{I(p=1)}}⋅\delta^{t-k}⋅ \frac{e^\gamma}{\phi⋅\gamma} $$

The last term is a two parameter power cost function, $I(k=t)$ is an indicator function equal to one if the decision occurs in the same period as the effort, and $I(p=1)$ is an indicator that the decision is a prediction, $\beta_h$ is the perceived present bias parameter (that is, the agent's degree of awareness of his present bias), and $\delta$ is the standard time discounting parameter. 

Taking the derivative of the maximization problem above with respect to effort yields the following first order condition:

$$  e^*= \left(\frac{\delta^{T-k}⋅\phi⋅w}{\frac{1}{\beta^{I(k=t)}}⋅\frac{1}{\beta_h^{I(p=1)}}⋅\delta^{t-k}} \right)^{\frac{1}{\gamma-1}} $$

This is the optimal effort level, or what we will call in the code the predicted choice. 

To model heterogeneity, the authors assume that the observed effort is distributed as the predicted effort plus an implementation error which is Gaussian with mean zero and standard deviation $\sigma$, so that the likelihood of observing an effort decision $e_j$ in the data is equal to:

$$ L(e_j)= \phi \left(\frac{e^*_j-e_j}{\sigma}\right)$$

where $\phi$ is the pdf of a standard normal. 

To deal with corner solutions we apply a Tobit correction, so that the likelihood to maximize is:

$$ 
\begin{aligned}
L^{tobit}(e_j) &=
    \underbrace{(1-I(e=10)-I(e=110))⋅\phi \left(\frac{e^*_j-e_j}{\sigma}\right)}_{\text{interior choice}} \\
    &+ \underbrace{I(e=10)⋅\left(1- \Phi \left(\frac{e_j^*-10}{\sigma}\right)\right)}_{\text{lower boundary}} \\
    &+ \underbrace{I(e=110)⋅ \Phi \left(\frac{e_j^*-110}{\sigma}\right)}_{\text{upper boundary}}
\end{aligned}    
$$

where $\Phi(\bullet)$ is the cdf of a standard normal, while $I(e=10)$ and $I(e=110)$ are the indicators `ind_effort10` and `ind_effort110` explained above. 

Note that, to keep the code simple, in this notebook we call effort the number of tasks performed by the agent, that is, the number of tasks chosen by the agent (ranging between 0 and 100) plus the compulsory 10 tasks. In the paper, the authors call effort just the number of tasks chosen by the agent (and, thus, they add 10 tasks to get total effort). This explains the differences between the equations in this notebook and equation (7), (8) and (10) in Section 3 of the paper. 

Our goal is to minimize the negative of the sum of the logarithms of $L^{tobit}$.

In [6]:
def negloglike(
        params: anp.ndarray, 
        netdistance: anp.ndarray, 
        wage: anp.ndarray, 
        today: anp.ndarray, 
        prediction: anp.ndarray, 
        pb: anp.ndarray, 
        effort: anp.ndarray, 
        ind_effort10: anp.ndarray, 
        ind_effort110: anp.ndarray
) -> float:
    """
    Computes the negative of the log likelihood of observing our data given the parameters of the model.

    Parameters
    ----------
    params : np.ndarray
        Vector of model parameters: beta, betahat, delta, gamma, alpha, sigma
    netdistance : np.ndarray 
        Stores (T-k)-(t-k) = T-t, the difference between the payment date T and the work time t
    wage : np.ndarray
        Stores the amount paid per task in a certain session
    today : np.ndarray 
        Dummy variable equal to one if the decision involves the choice of work today
    prediction : np.ndarray
        Dummy variable equal to one if the decision involves the choice of work in the future
    pb : np.ndarray
        Dummy equal to one if the subject completed 10 mandatory tasks on subject-day 
    effort : np.ndarray
        Number of tasks completed by a subject in a session. It can range from a minimum of 10 to a maximum of 110
    ind_effort10 : np.ndarray 
        Dummy equal to one if the subject's effort was equal to 10
    ind_effort110 : np.ndarray 
        Dummy equal to one if the subject's effort was equal to 110

    Returns
    -------
    float
        Value of negative log likelihood
    """
    
    beta, betahat, delta, gamma, phi, alpha, sigma = params

    # predchoice is the predicted choice coming from the optimality condition of the subject
    
    predchoice=((phi*(delta**netdistance)*(beta**today)*(betahat**prediction)*wage)**(1/(gamma-1)))-pb*alpha
    
    # prob is a 1x8049 vector containing the probability of observing the effort of an individual. 
    # If effort is 10 or 110 we apply a Tobit correction
    prob = (1-ind_effort10-ind_effort110)*norm.pdf(effort, predchoice, sigma) \
        +ind_effort10*(1 - norm.cdf((predchoice-effort)/sigma)) \
        +ind_effort110*norm.cdf((predchoice-effort)/sigma)
            
    # Add a small value close to zero if prob=0 or subtract a small value close to zero if prob=1. 
    # This is necessary to avoid problems when taking logs
        
    prob = anp.where(prob == 0.0, 1.0e-4, prob)
    prob = anp.where(prob == 1.0, 1.0 - 1.0e-4, prob)
    
    negll = - anp.sum(anp.log(prob)) # negative log likelihood
    
    return negll

## 3. Estimation

### Point Estimates

We now estimate the model. First, we need to initialize a vector with the starting parameters for the minimization algorithm. We then minimize the negative log-likelihood function using the scipy.optimize package and the Nelder-Mead algorithm.

In [7]:
# Define the initial guesses (same as the ones used by the authors in their do.file) 
# and the arguments for the function to minimize

# starting parameters for the algorithm

beta_init = 0.8         # Present-bias discounting
betahat_init = 1.0      # Perceived discounting
delta_init = 1.0        # Discount factor
gamma_init = 2.0        # Cost curvature
phi_init = 500.0        # Preference parameter
alpha_init = 7.0        # Projected task reduction
sigma_init = 40.0       # Variance of Gaussian noise

par_init = anp.array([beta_init, betahat_init, delta_init, gamma_init, phi_init, alpha_init, sigma_init])

# Additional arguments: split DataFrame columns into individual arrays. 
# Order matters!
columns = ['netdistance', 'wage', 'today', 'prediction', 'pb', 'effort', 'ind_effort10', 'ind_effort110']
mle_args = tuple(anp.array(dt[name].to_numpy()) for name in columns)

# we now find the estimates using the scipy.optimize package

sol = opt.minimize(negloglike, par_init, args=mle_args, method='Nelder-Mead', options={'maxiter': 1500})
res = sol.x

In [8]:
# Print estimated parameters
parameters_name = [
    "Present Bias β",
    "Naive Pres. Bias β_h",
    "Discount Factor δ",
    "Cost Curvature γ",
    "Cost Slope ϕ",
    "Proj Task Reduction α",
    "Sd of error term σ"
]

print(pd.Series(res, index=parameters_name))

Present Bias β             0.834996
Naive Pres. Bias β_h       0.999058
Discount Factor δ          1.002621
Cost Curvature γ           2.145292
Cost Slope ϕ             724.145634
Proj Task Reduction α      7.303662
Sd of error term σ        42.621433
dtype: float64


### Standard Errors

We now estimate individual cluster robust standard errors. 

These are computed by taking the square root of the diagonal elements of the following matrix: 

$$ Adj⋅(H^{-1} \cdot G \cdot H^{-1}) $$ 

Where Adj is an adjustment for the degree of freedoms and the number of clusters:

$$ Adj = \frac{Nr.observations-1}{Nr.observations-Nr.parameters}⋅\frac{Nr.clusters}{Nr.clusters-1} $$ 

$H^{-1}$ is the inverse of the Hessian of the negative log-likelihood evaluated in the minimum (our estimates), and $G$ is a $5\times 5$ matrix of gradient contributions. 

We denote the gradient of the log likelihood function for a generic individual $i$ as follows:

$$  g_i(y|\theta) = [log f_i(y|\theta)]' = \frac{\partial}{\partial \theta} log f_i(y|\theta) $$

where $\theta$ is the parameters vector and $f_i(y|\theta)$ is the likelihood function. Then $G$ is defined as follows:

$$ G = \sum_j \left[\sum_{i \in c_j}g_i(y|\hat{\theta})\right]^{\top}\left[\sum_{i \in c_j}g_i(y|\hat{\theta})\right] $$

where $J$ is the number of clusters (in our case the number of unique individuals = 72) and $c_j$ is a generic cluster $j$, that includes all observations for a specific individual (in our case 130). For more information on how to compute standard errors when using maximum likelihood, we refer the reader to David A. Freedman, 2006, ["On The So-Called 'Huber Sandwich Estimator' and 'Robust Standard Errors'"](https://snunnari.github.io/freedman.pdf), *The American Statistician*, 60:4, 299-302).

In [9]:
# Define gradient and Hessian functions using autograd
Hfun = autograd.hessian(negloglike)
gradfun = autograd.grad(negloglike)

In [10]:
# Compute the Hessian
hessian = Hfun(res, *mle_args)      # hessian
hess_inv = np.linalg.inv(hessian)   # inverse of the hessian

In [11]:
# Compute gradient contribution for each individual 
# (using apply along axis=1)

gradients = dt.apply(
    lambda x: gradfun(res, *tuple(x[name] for name in columns)), 
    axis=1,
    result_type='expand'
)

In [12]:
# Add individual ID, sum within individual
gradients.index = pd.Index(dt["wid"])
gradients_indiv = gradients.groupby("wid").sum()

In [13]:
# Compute outer product for each individual
gradients_indiv = gradients_indiv.to_numpy()
# This creates an Nindiv x 5 x 5 array
G_j = gradients_indiv[:, :, None] * gradients_indiv[:, None, :]

# Sum over individuals to get final 5 x 5 matrix
G = np.sum(G_j, axis=0)

In [14]:
# Compute the cluster robust standard errors
Nobs = len(dt)
Nindiv = len(dt['wid'].unique())
# Number of parameters
K = len(res)

# Compute the adjustment for degree of freedoms and number of clusters
adj = (Nobs-1)/(Nobs-K) * Nindiv/(Nindiv-1)

varcov_estimates = adj *(hess_inv @ G @ hess_inv)   # var-cov matrix of our estimates
se_cluster = np.sqrt(np.diag((varcov_estimates)))   # individual cluster robust standard errors

### Hypothesis Testing

We now do some hypothesis testing on the parameters we obtained. We compute the $z$-test statistics and the corresponding $p$-values to check if $\beta$, $\beta_h$ or $\delta$ are statistically different from one. We then compute the $p$-value of a $z$-test to check if the parameter for projection bias is statistically different from zero.

In [15]:
# Compute the z-test statistics and the corresponding p-values to check if beta, betahat, delta are statistically different from one

zvalues_1 = (np.array(res[0:3])-1)/np.array(se_cluster[0:3]) # the first three elements are for beta (position 0), betahat (position 1) and delta (position 2)
pvalues_1 = 2*(1-norm.cdf(np.abs(zvalues_1),0,1))

# Now compute the z-test statistics and the corresponding p-value for H0: alpha different from 0

zvalue_a = res[5]/se_cluster[5]
pvalue_a = 2*(1-norm.cdf(np.abs(zvalue_a),0,1))

## 4. Print and Save Estimation Results

We create a table with point estimates and individual cluster robust standard errors. We then save the results as a csv file in the output folder and print the results. This replicates Column 1 of Table 1 in the paper.

In [16]:
# Create a new DataFrame with the results and save it as a csv file in output. We round the results up to the 3rd decimal.

Table_1 = pd.DataFrame({
    'parameters': parameters_name,
    'estimates': np.round(res, 3),
    'standarderr': np.round(se_cluster,3)
})

Table_1.to_csv('../output/table1_python.csv', index=False)

In [17]:
# Print the results

from IPython.display import display

print("Table 1: Primary aggregate structural estimation")
display(Table_1)
print("Number of observations:", f"{Nobs:,}")
print("Number of participants:", f"{Nindiv:,}")
print("Log Likelihood:", f"{-sol.fun:,.0f}")
print("H_0(β=1)", f"{pvalues_1[0]:,.2f}")
print("H_0(β_h=1):", f"{pvalues_1[1]:,.2f}")
print("H_0(α=0):", f"{pvalue_a:,.3f}")
print("H_0(δ=1):", f"{pvalues_1[2]:,.2f}")

Table 1: Primary aggregate structural estimation


,parameters,estimates,standarderr
0,Present Bias β,0.835,0.038
1,Naive Pres. Bias β_h,0.999,0.011
2,Discount Factor δ,1.003,0.003
3,Cost Curvature γ,2.145,0.071
4,Cost Slope ϕ,724.146,255.711
5,Proj Task Reduction α,7.304,2.599
6,Sd of error term σ,42.621,3.305


Number of observations: 8,049
Number of participants: 72
Log Likelihood: -28,412
H_0(β=1) 0.00
H_0(β_h=1): 0.93
H_0(α=0): 0.005
H_0(δ=1): 0.37
